In [1]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np

import sys
sys.path.append('c:\\users\\jacob\\onedrive\\documents\\college\\rice university\\cmor492-dws\\')
from network_construction.network import source_treatment, get_Utown
import json

In [2]:
G = get_Utown()
source_nodes, treatment_nodes = source_treatment(G, 40)  # <-- Specify # starting points for treatment node algorithm

In [3]:
### MODEL PARAMETERS

# TODO: What if we just make pipe size continuous/linear

Path = {}  # Set of shortest paths from each source node i to each treatment node j
NLinks = {}  # Number of edges in each path
L = {}  # Length of each path (distance)

for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='length')
        Path[i, j] = path
        NLinks[i, j] = len(path)-1
        L[i, j] = nx.path_weight(G, path, weight='length')
LE = {e: G.edges[e]['length'] for e in G.edges}  # Length of edge e
EL = {v: G.nodes[v]['elevation'] for v in G.nodes}  # Elevation of node v


D = [0.2, 0.25, 0.3, 0.35, 0.40, 0.45]  # Pipe diameters
CP = {0.05: 8.7, 0.06: 9.5, 0.08: 11,
                       0.1: 12.6, 0.15: 43.5, 0.2: 141,
                       0.25: 151, 0.3: 161, 0.35: 180,
                       0.4: 190, 0.45: 200}  # Cost per unit of pipe


SR = {}  # Production at Source node i
CAP = {}  # Capacity at treatment node j

for node in source_nodes:
    G.nodes[node]['production'] = .17 * 2
    SR[node] = .17 * 2

total_flow = sum(SR.values())

for node in treatment_nodes:
    G.nodes[node]['capacity'] = 100
    CAP[node] = 100

Vmin = 0.6 * 60
Vmax = 3 * 60

CE = 25  # Cost of Excavation
CB = 6  # Cost of Bedding
TR = 44000  # Fixed Cost of Treatment Plant
TRFlow = 100  # Variable Cost of Treatment
PICost = 30

PF = {'0.05': 8.7, '0.06': 9.5, '0.08': 11,
                       '0.1': 12.6, '0.15': 43.5, '0.2': 141,
                       '0.25': 151, '0.3': 161, '0.35': 180,
                       '0.4': 190, '0.45': 200}  # Fixed Cost of Piping

CT = 1000000000  # Cost of trucking
M = 1e6

Smin = 0.01
Smax = 0.1
W = 0.5  # Buffer Width

T = [0, 1]

In [4]:
import ast

### T=0 PARAMETERS

with (open("x_sol.json", "r") as f):
    x_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}

with open("y_sol.json", "r") as f:
    y_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}

with open("z_sol.json", "r") as f:
    z_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}

with open("d_sol.json", "r") as f:
    d_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}

with open("el_sol.json", "r") as f:
    el_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}


In [5]:
m = gp.Model()

x = m.addVars(Path.keys(), T, vtype=GRB.BINARY, name='x')  # Path ij used
y = m.addVars(treatment_nodes, T, vtype=GRB.BINARY, name='y')  # treatment at node j
z = m.addVars(G.edges, T, vtype=GRB.BINARY, name='z')  # edge e used

d = m.addVars(G.edges, D, T, vtype=GRB.BINARY, name='d')  # Pipe size s at edge e

a = m.addVars(G.edges, D, vtype=GRB.BINARY, name='a')

r = m.addVars(source_nodes, vtype=GRB.CONTINUOUS, lb=0.0, name='r')  # flow handled at trucking at edge e

Q = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb=0.0, name='Q')  # Flow in Edge e

el = m.addVars(G.nodes, T, vtype=GRB.CONTINUOUS, name='el')  # Elevation at node el_v

p = m.addVars(Path.keys(), vtype=GRB.CONTINUOUS, lb = 0.0, name='p')

c = m.addVars(G.nodes, vtype=GRB.BINARY, name='c')

m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-08


In [6]:
x_0_c = m.addConstrs((x[i, j, 0] == x_0[i, j] for i, j in Path.keys()), name='x_0')

y_0_c = m.addConstrs((y[j, 0] == y_0[j] for j in treatment_nodes), name='y_0')

z_0_c = m.addConstrs((z[*e, 0] == z_0[e] for e in G.edges), name='z_0')

d_0_c = m.addConstrs((d[*e, s, 0] == d_0[*e, s] for e in G.edges for s in D), name='d_0')

el_0_c = m.addConstrs((el[u, 0] == el_0[u] for u in G.nodes), name='el_0')

In [7]:
### CONSTRAINTS

# NODE PRODUCTION MINUS RECOURSE
node_prod_rec = m.addConstrs((p[i, j] >= (SR[i] * x[i, j, 1]) - r[i] for i, j in Path.keys()), name='node_prod_rec')


# TREATMENT CAPACITY
treat_cap = m.addConstrs((gp.quicksum(p[i, j] for i in source_nodes) <= CAP[j] * y[j, 1] for j in treatment_nodes), name='treat_cap')

#  NODE ASSIGNMENT
node_assign = m.addConstrs((gp.quicksum(x[i, j, 1] for j in treatment_nodes) == 1 for i in source_nodes), name='node_assign')

# EDGE ACTIVATION
ePath = {}  # Use this for Edge Activiation Constraint
for e, p_ in Path.items():
    ePath[e] = [(p_[l - 1], p_[l]) for l in range(1, len(p_))]

edge_activate = m.addConstrs((gp.quicksum(z[*e, 1] for e in ePath[i, j]) >= NLinks[i, j] * x[i, j, 1] for i, j in Path), name='edge_activate')

# FLOW DEFINITION
def is_sublist(short_list, long_list):
    for i in range(len(long_list) - len(short_list) + 1):
        if long_list[i:i + len(short_list)] == short_list:
            return True
    return False

flow_def = m.addConstrs((Q[e] == gp.quicksum(p[i, j] for i, j in Path.keys() if is_sublist(list((e[0], e[1])),Path[i,j])) for e in G.edges), name='flow_def')


In [8]:
# PIPE SIZING
pipe_sizing1 = m.addConstrs((gp.quicksum(a[*e, s] for s in D) == z[*e, 1] for e in G.edges), name='pipe_size_a')
pipe_sizing2 = m.addConstrs((gp.quicksum(d[*e, s, 1] for s in D) <= 1 for e in G.edges), name='pipe_sizing_d')

# a-Constraint
a_constraint = m.addConstrs((a[*e, s] == d[*e, s, 1] + d_0[*e, s] for e in G.edges for s in D), name='a_constraint')

# NODE ELEVATION CHANGE
node_elevation1 = m.addConstrs((gp.quicksum(d[*e, s, 1] for s in D) >= (0.5 * (c[e[0]] - c[e[1]])) + (z[*e, 1] - 1) for e in G.edges), name='node_elevation1')
cu_cons1 = m.addConstrs((c[u] >= (el[u, 1] - el_0[u]) / M for u in G.nodes), name='cu_cons1')
cu_cons2 = m.addConstrs((c[u] >= (el_0[u] - el[u, 1]) / M for u in G.nodes), name='cu_cons2')

# MIN/MAX SLOPE
min_slope = m.addConstrs((el[e[0], 1] - el[e[1], 1] >= (LE[e] * Smin) - (M * (1 - z[*e, 1])) for e in G.edges), name='min_slope')
max_slope = m.addConstrs((el[e[0], 1] - el[e[1], 1] <= (LE[e] * Smax) + (M * (1 - z[*e, 1])) for e in G.edges), name='max_slope')

# FLOW VELOCITY LIMIT
flow_vel = m.addConstrs((Q[e] <= Vmax * gp.quicksum((np.pi / 8) * (s**2) * (a[*e, s]) for s in D) for e in G.edges), name='flow_vel')

# BELOW GROUND PIPES
underground = m.addConstrs((el[u, 1] <= EL[u] for u in G.nodes), name='underground')

# TREATMENT CONTINUITY
treatment_cont = m.addConstrs((y_0[j] <= y[j, 1] for j in treatment_nodes), name='treatment_cont')

# ENVELOPES FOR MANNING

T_ = 11.9879
P = lambda LE, s: LE / (T_ * (s**(16/3)))
Qmax = lambda s: Vmax * ((np.pi / 8) * (s**2))


alpha = m.addVars(G.edges, D, lb=0, name='alpha')
beta = m.addVars(G.edges, D, lb=0, name='beta')


alpha_2 = m.addConstrs((alpha[*e, s] >= Q[e] + a[*e, s] * Qmax(s) - ( Qmax(s)) for e in G.edges for s in D), name='alpha_2')
alpha_3 = m.addConstrs((alpha[*e, s] <= Qmax(s) * a[*e, s] for e in G.edges for s in D), name='alpha_3')
alpha_4 = m.addConstrs((alpha[*e, s] <= Q[e] for e in G.edges for s in D), name='alpha_4')
alpha_5 = m.addConstrs((alpha[*e, s] <= Qmax(s) for e in G.edges for s in D), name='alpha_5')

beta_2 = m.addConstrs((beta[*e, s] >= (Qmax(s) * Q[e]) + (Qmax(s) * alpha[*e, s]) - (Qmax(s)**2) for e in G.edges for s in D), name='beta_2')
beta_3 = m.addConstrs((beta[*e, s] <= Qmax(s) * alpha[*e, s] for e in G.edges for s in D), name='beta_3')
beta_4 = m.addConstrs((beta[*e, s] <= Qmax(s) * Q[e] for e in G.edges for s in D), name='beta_4')

manning_2 = m.addConstrs((el[e[1], 1] - el[e[0], 1] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= (1-z[*e, 1]) * M for e in G.edges), name='manning_2')
m.update()

In [9]:
# OBJECTIVE EPXR 1: TREATMENT COSTS

treat_cost = gp.LinExpr()
for j in treatment_nodes:
    treat_cost.addTerms(TR, y[j, 1])
    treat_cost.addTerms(-1*TR, y[j, 0])
    for i in source_nodes:
        treat_cost.addTerms(TRFlow * SR[i], x[i, j, 1])


# OBJECTIVE EXPR 2: EXCAVATION COSTS
excav_cost_f = lambda u, v: gp.QuadExpr(CE * (((EL[u] - el[u, 1]) + (EL[v] - el[v, 1])) / 2) * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s, 1]) for s in D))



# OBJECTIVE EXPR 3: BEDDING COSTS
bed_cost_f = lambda u, v: gp.LinExpr(CB * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s, 1]) for s in D))


# OBJECTIVE EXPR 4: PIPE COSTS
pipe_cost_f = lambda u, v: gp.LinExpr(LE[u, v] * gp.quicksum(CP[s] * d[u, v, s, 1] for s in D))

excav_bed_cost = gp.quicksum(excav_cost_f(u, v) + bed_cost_f(u, v) + pipe_cost_f(u, v) for u, v in G.edges)



# OBJECTIVE EXPR 5: RECOURSE TRUCKING
rec_cost = gp.LinExpr()
for i in source_nodes:
    rec_cost.addTerms(CT, r[i])

m.setObjective(treat_cost + excav_bed_cost + rec_cost, GRB.MINIMIZE)

print(f"Model has {m.NumVars} variables and {m.NumConstrs} constraints.")
# m.setObjective(0, GRB.MINIMIZE)
m.update()

Model has 38940 variables and 54336 constraints.


In [55]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 54336 rows, 38940 columns and 311717 nonzeros
Model fingerprint: 0xddb8f2dc
Model has 6054 quadratic objective terms
Variable types: 14259 continuous, 24681 integer (24681 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+06]
  Objective range  [3e+01, 1e+09]
  QObjective range [3e+02, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-05, 1e+06]
Presolve removed 48774 rows and 34159 columns
Presolve time: 1.32s
Presolved: 7890 rows, 7109 columns, 32845 nonzeros
Variable types: 4682 continuous, 2427 integer (2427 binary)
Found heuristic solution: objective 9.453271e+09

Root relaxation: objective 2.919912e+08, 2129 iterations, 0.16 seconds (0.06 work units)

    Nodes    |    Current Node    |     O

In [11]:
m.computeIIS()

for c in m.getConstrs():
    if c.IISConstr:
        print(f"Constraint {c.ConstrName} is in the IIS")

for v in m.getVars():
    if v.IISLB > 0:
        print(f"Lower bound of {v.VarName} is in the IIS")
    elif v.IISUB > 0:
        print(f"Upper bound of {v.VarName} is in the IIS")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

IIS computation: initial model status unknown, solving to determine model status
Presolve removed 51033 rows and 36452 columns
Presolve time: 0.55s
Presolved: 3303 rows, 2488 columns, 12326 nonzeros
Variable types: 2062 continuous, 426 integer (426 binary)
Found heuristic solution: objective 3.543946e+08

Explored 0 nodes (0 simplex iterations) in 0.73 seconds (0.28 work units)
Thread count was 20 (of 20 available processors)

Solution count 1: 3.54395e+08 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.543945645197e+08, best bound 3.543945645197e+08, gap 0.0000%
IIS runtime: 0.75 seconds (0.28 work units)


GurobiError: Cannot compute IIS on a feasible model

In [26]:
import processing


In [57]:
modelo = processing.ModelOutput(m)

print(modelo.solution)
m.Status

None


2

In [61]:
class BuiltDiff:
    """
    ModelOutput will take in a solved GurobiPy model and parse its output effectively.

    The special purpose will involve passing context from an earlier model to a later model in multistage settings.
    """

    def __init__(self, model: gp.Model):
        self.model = model
        self.infeasible = model.status == gp.GRB.INFEASIBLE
        self.solution = None
        self.extracted = False

        self.orig_hold = None

    def __str__(self):
        return "I'm a model!"
    
    def extract_solution(self, x, y, z, d, el):
        """
        Extract and store the solution details from the model.
        """
        if self.model.status != gp.GRB.INFEASIBLE:
            raw_output = self.model.getVars()

            self.solution = {var.varName: var.getAttr(GRB.Attr.X) for var in raw_output}
        else:
            self.solution = None
            print("Model is not feasible. No solution to extract.")

        self.extracted = True

In [62]:
go_hard = BuiltDiff(m)


go_hard.extract_solution(x, y, z, d, el)
print(go_hard.solution)

{'x[59079984,59128065,0]': 0.0, 'x[59079984,59128065,1]': 0.0, 'x[59079984,59133314,0]': 0.0, 'x[59079984,59133314,1]': 0.0, 'x[59079984,59116291,0]': 0.0, 'x[59079984,59116291,1]': 0.0, 'x[59079984,59121412,0]': 0.0, 'x[59079984,59121412,1]': 0.0, 'x[59079984,59081605,0]': 0.0, 'x[59079984,59081605,1]': 0.0, 'x[59079984,59126150,0]': 0.0, 'x[59079984,59126150,1]': 0.0, 'x[59079984,59133192,0]': 0.0, 'x[59079984,59133192,1]': 0.0, 'x[59079984,59117577,0]': 0.0, 'x[59079984,59117577,1]': 0.0, 'x[59079984,59091084,0]': 0.0, 'x[59079984,59091084,1]': 0.0, 'x[59079984,59123853,0]': 0.0, 'x[59079984,59123853,1]': 0.0, 'x[59079984,59110286,0]': 0.0, 'x[59079984,59110286,1]': 0.0, 'x[59079984,59116300,0]': 0.0, 'x[59079984,59116300,1]': 0.0, 'x[59079984,59129101,0]': 0.0, 'x[59079984,59129101,1]': 0.0, 'x[59079984,59146899,0]': 0.0, 'x[59079984,59146899,1]': 0.0, 'x[59079984,59095700,0]': 0.0, 'x[59079984,59095700,1]': 0.0, 'x[59079984,59124248,0]': 0.0, 'x[59079984,59124248,1]': 0.0, 'x[5907

In [64]:
softer = ModelOutput(m)


softer.extract_solution()
print(softer.solution)

softer.add_context(m)

{'x[59079984,59128065,0]': 0.0, 'x[59079984,59128065,1]': 0.0, 'x[59079984,59133314,0]': 0.0, 'x[59079984,59133314,1]': 0.0, 'x[59079984,59116291,0]': 0.0, 'x[59079984,59116291,1]': 0.0, 'x[59079984,59121412,0]': 0.0, 'x[59079984,59121412,1]': 0.0, 'x[59079984,59081605,0]': 0.0, 'x[59079984,59081605,1]': 0.0, 'x[59079984,59126150,0]': 0.0, 'x[59079984,59126150,1]': 0.0, 'x[59079984,59133192,0]': 0.0, 'x[59079984,59133192,1]': 0.0, 'x[59079984,59117577,0]': 0.0, 'x[59079984,59117577,1]': 0.0, 'x[59079984,59091084,0]': 0.0, 'x[59079984,59091084,1]': 0.0, 'x[59079984,59123853,0]': 0.0, 'x[59079984,59123853,1]': 0.0, 'x[59079984,59110286,0]': 0.0, 'x[59079984,59110286,1]': 0.0, 'x[59079984,59116300,0]': 0.0, 'x[59079984,59116300,1]': 0.0, 'x[59079984,59129101,0]': 0.0, 'x[59079984,59129101,1]': 0.0, 'x[59079984,59146899,0]': 0.0, 'x[59079984,59146899,1]': 0.0, 'x[59079984,59095700,0]': 0.0, 'x[59079984,59095700,1]': 0.0, 'x[59079984,59124248,0]': 0.0, 'x[59079984,59124248,1]': 0.0, 'x[5907

In [65]:
softer.orig_hold

{'x': {'x[59079984,59128065,0]': 0.0,
  'x[59079984,59128065,1]': 0.0,
  'x[59079984,59133314,0]': 0.0,
  'x[59079984,59133314,1]': 0.0,
  'x[59079984,59116291,0]': 0.0,
  'x[59079984,59116291,1]': 0.0,
  'x[59079984,59121412,0]': 0.0,
  'x[59079984,59121412,1]': 0.0,
  'x[59079984,59081605,0]': 0.0,
  'x[59079984,59081605,1]': 0.0,
  'x[59079984,59126150,0]': 0.0,
  'x[59079984,59126150,1]': 0.0,
  'x[59079984,59133192,0]': 0.0,
  'x[59079984,59133192,1]': 0.0,
  'x[59079984,59117577,0]': 0.0,
  'x[59079984,59117577,1]': 0.0,
  'x[59079984,59091084,0]': 0.0,
  'x[59079984,59091084,1]': 0.0,
  'x[59079984,59123853,0]': 0.0,
  'x[59079984,59123853,1]': 0.0,
  'x[59079984,59110286,0]': 0.0,
  'x[59079984,59110286,1]': 0.0,
  'x[59079984,59116300,0]': 0.0,
  'x[59079984,59116300,1]': 0.0,
  'x[59079984,59129101,0]': 0.0,
  'x[59079984,59129101,1]': 0.0,
  'x[59079984,59146899,0]': 0.0,
  'x[59079984,59146899,1]': 0.0,
  'x[59079984,59095700,0]': 0.0,
  'x[59079984,59095700,1]': 0.0,
  'x[

In [21]:
linear = gp.Model()

x = linear.addVars(5, vtype=gp.GRB.CONTINUOUS, name="x")

linear.setObjective(4*x[0] + 2*x[1] + 3*x[2] + 4*x[3] + 2*x[4], gp.GRB.MAXIMIZE)
linear.addConstr(x[0] + 2*x[1] + x[2] <=8)
linear.addConstr(x[0] + 2*x[1] + 2*x[2] <=8)
linear.addConstr(x[3] + x[4] <=8)

linear.optimize()

print(linear.ObjVal)

for v in linear.getVars():
    print(v.varName, v.X)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 3 rows, 5 columns and 8 nonzeros
Model fingerprint: 0x3afa6657
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 8e+00]
Presolve removed 1 rows and 3 columns
Presolve time: 0.02s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4000000e+01   0.000000e+00   0.000000e+00      0s
       0    6.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds (0.00 work units)
Optimal objective  6.400000000e+01
64.0
x[0] 8.0
x[1] 0.0
x[2] 0.0
x[3] 8.0
x[4] 0.0


In [60]:
lin = BuiltDiff(linear)
lin.extract_solution()

TypeError: BuiltDiff.extract_solution() missing 5 required positional arguments: 'x', 'y', 'z', 'd', and 'el'

In [52]:
lin.solution

{'x[0]': 8.0, 'x[1]': 0.0, 'x[2]': 0.0, 'x[3]': 8.0, 'x[4]': 0.0}